In [1]:
import pandas as pd 
import emoji

In [2]:
gpt = pd.read_json('../../datasets/gptneo.json', lines=True)

In [3]:
gpt.drop("timestamp", axis=1, inplace=True)
gpt.drop("users", axis=1, inplace=True)

In [4]:
gpt.head(10)

,tweets,artificial
0,"I do hope #COP26 from/inGlasgow, is a great on...",0
1,GOVERNMENT BILL - STOP WALLING STRAIGHT BANGLA...,1
2,The @Conservatives,1
3,#cdnpoli #cdnmedia it would be fantastic if a ...,0
4,@TorontoStar @hscoffield I remember when we we...,1
5,and we said it is a blessing in disguise think...,0
6,@ianmSC @mattwridley @j_preparedmind @GretaThu...,1
7,"Scum. Yep, definitely scum. Greedy, money-grub...",0
8,@RichardDunda @CNBC While world leaders fly pr...,0
9,"""Climate change is a risk to national security...",0


In [5]:
#delete Emojis
def delete_emojis(text):
    return emoji.demojize(text)


gpt["tweets"] = gpt["tweets"].apply(delete_emojis)


In [6]:
#remove URL
gpt['tweets'] = gpt['tweets'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


In [7]:
#remove taggings    
gpt['tweets'] = gpt['tweets'].str.replace(r'@\S+', '', regex=True)

In [8]:
#remove #
gpt['tweets'] = gpt['tweets'].str.replace(r'#\S+', '', regex=True)


In [9]:
#remove quotes
gpt['tweets'] = gpt['tweets'].str.replace(r'"', '', regex=True).replace(r"'", '', regex=True)

In [10]:
#remove every special characters
gpt['tweets'] = gpt['tweets'].str.replace(r'[^\w\s]', '', regex=True)

In [11]:
#remove numbers
gpt['tweets'] = gpt['tweets'].str.replace(r'\d+', '', regex=True)

In [12]:
#lowercase
gpt['tweets'] = gpt['tweets'].str.lower()

In [13]:
#remove stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

gpt["tweets"] = gpt["tweets"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#lemmatize everything
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.tag import pos_tag

# Download necessary ntkl data files
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]
    return ' '.join(lemmatized_tokens)

gpt["tweets"] = gpt["tweets"].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
#change column name
gpt.rename(columns={"tweets": "text"}, inplace=True)
gpt.rename(columns={"artificial": "label"}, inplace=True)

In [16]:
#writing to csv
gpt.to_csv('../StorePreprocessed/GPTNeocsv.csv', index=False)